# Polish Crops analysis

With this notebook I would like to playaround some machine learning Python tools. 

Therefore I've decided to create this simple project, in which I want to check a simple hypothesis - is percipitation and/or temperature in Poland affecting prices of crops (soft wheat). In order to achieve it here are the steps:

* **Step 1 Download source reports** - downloading necessary files from publically available databases.

* **Step 2 Data preprocessing** - unnecessary data are removed and prepared for further processing.

* **Step 3 Data visualization** - plotting couple of charts, just to visualize the downloaded data.

* **Step 4 Train machine learning models** - training couple of machine learning regression models to check which one of them fits the best - is it a correlaction between climate (temperature, percipitation) and prices of soft wheat.
---

## Step 1 Download source reports

First you need to download necessary data. They are downloaded from following sources:

* [Eurostat](https://ec.europa.eu/eurostat/data/database)
* [IMGW (Instytut Meteorologii i Gospodarki Wodnej, *eng.* Polish Institute of Meteorology and Water Management)](https://danepubliczne.imgw.pl)

## Step 2 Data preprocessing


## Step 3 Data visualization

## Step 4 Train machine learning models